# NASA Mars News

In [1]:
#import dependencies
import pandas as pd
import requests
from splinter import Browser
import os
from bs4 import BeautifulSoup as bs
import pymongo

In [2]:
# URL of page to be scraped
nasa_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
# Retrieve page with the requests module
response = requests.get(nasa_url)
soup = bs(response.text, 'lxml')
result = soup.body.find('div', class_="slide")
result

<div class="slide">
<div class="image_and_description_container">
<a href="/news/8348/opportunity-hunkers-down-during-dust-storm/">
<div class="rollover_description">
<div class="rollover_description_inner">
It's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or even months, before skies are clear enough for NASA's Opportunity rover to recharge its batteries and phone home. 
</div>
<div class="overlay_arrow">
<img alt="More" src="/assets/overlay-arrow.png"/>
</div>
</div>
<img alt="Opportunity Hunkers Down During Dust Storm" class="img-lazy" data-lazy="/system/news_items/list_view_images/8348_PIA22521-320.jpg" src="/assets/loading_320x240.png"/>
</a>
</div>
<div class="content_title">
<a href="/news/8348/opportunity-hunkers-down-during-dust-storm/">
Opportunity Hunkers Down During Dust Storm
</a>
</div>
</div>

In [3]:
# Assign the text to variables
news_title = result.find('div',class_='content_title').text
news_p = result.find('div',class_='rollover_description_inner').text
print(news_title)
print(news_p)



Opportunity Hunkers Down During Dust Storm



It's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or even months, before skies are clear enough for NASA's Opportunity rover to recharge its batteries and phone home. 



# JPL Mars Space Images - Featured Image

In [4]:
#set up 
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

jpl_html = browser.html
jpl_soup = bs(jpl_html, 'lxml')

In [5]:
# Finding the url for image
jpl_result = jpl_soup.find('div', class_='default floating_text_area ms-layer')
featured_img = jpl_result.find('footer')
featured_img_url = 'https://www.jpl.nasa.gov'+ featured_img.find('a')['data-fancybox-href']
print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA01384_ip.jpg


# Mars Weather

In [6]:
twitter_url = 'https://twitter.com/marswxreport?lang=en'

twitter_response = requests.get(twitter_url)
twitter_soup = bs(twitter_response.text, 'lxml')

twitter_result = twitter_soup.find('div', class_='js-tweet-text-container')

mars_weather = twitter_result.find('p', class_='js-tweet-text').text
mars_weather

'Sol 2230 (2018-11-14), high -5C/23F, low -72C/-97F, pressure at 8.59 hPa, daylight 06:22-18:39'

# Mars Facts

In [16]:
fact_url = 'https://space-facts.com/mars/'
fact_table = pd.read_html(fact_url)[0]
fact_table.columns = ['Description','Value']
fact_table.set_index('Description', inplace=True)
fact_table

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [18]:
# Export pandas df to html script
fact_html = fact_table.to_html()
fact_html.replace("\n", "")
fact_table.to_html('mars_fact.html')

# Mars Hemispheres

In [19]:
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_url)

usgs_html = browser.html
usgs_soup = bs(usgs_html, 'lxml')
base_url ="https://astrogeology.usgs.gov"

images_list = usgs_soup.find_all('div', class_='item')

image_urls  = []

# Loop through each result and add it to the list 
for image in images_list:
    usgs_dict = {}
    
    href = image.find('a', class_='itemLink product-item')
    link = base_url + href['href']
    browser.visit(link)
    
    hemisphere_html = browser.html
    hemisphere_soup = bs(hemisphere_html, 'lxml')
    
    title = hemisphere_soup.find('div', class_='content').find('h2', class_='title').text
    usgs_dict['title'] = title
    
    img_url = hemisphere_soup.find('div', class_='downloads').find('a')['href']
    usgs_dict['url_img'] = img_url
    
    image_urls.append(usgs_dict)
      
image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'url_img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'url_img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'url_img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'url_img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]